# LFT: Dynamics & Time (N=4→5)

In Logic Field Theory (LFT), **time** is the directed application of the logical operator \(L\) that reduces inconsistency and increases order. This notebook formalizes time as a **Lyapunov descent** on total orders (permutations) and extends it to partial orders:

1. Define the **inversion count** \(h(\sigma)\) on \(S_N\).
2. Prove that **adjacent swaps** that resolve inversions **strictly decrease** \(h\).
3. Simulate the descent for **N=4** and **N=5** from random and worst-case starts.
4. Extend to **partial orders** with a potential \(\tilde H\), verified on a concrete DAG example (N=4).

## 1. Inversion count and the bubble-sort lemma

**Definition.** For a permutation \(\sigma\in S_N\), the **inversion count**
$$h(\sigma)=\#\{(i,j)\mid 1\le i<j\le N,\ \sigma(i) > \sigma(j)\}$$
is the Kendall–Tau distance to the identity.

**Lemma (Bubble-sort step).** If an update performs an **adjacent swap** on indices \(k,k{+}1\) **only when** \(\sigma(k) > \sigma(k{+}1)\), then \(h\) decreases by exactly 1.

**Proof.** Only the pair \((k,k{+}1)\) changes relative order; if it was an inversion, swapping removes it and introduces no new inversions. ∎

In [ ]:
import numpy as np
import random
import os

# Ensure outputs directory exists
os.makedirs('./outputs', exist_ok=True)

def inversion_count(perm):
    """Count inversions in permutation - Kendall-Tau distance to identity"""
    inv = 0
    for i in range(len(perm)):
        for j in range(i+1, len(perm)):
            if perm[i] > perm[j]:
                inv += 1
    return inv

def local_descent_step(perm):
    """Pick a random adjacent pair; swap if it resolves an inversion."""
    N = len(perm)
    k = np.random.randint(0, N-1)
    p = list(perm)
    if p[k] > p[k+1]:  # Found an inversion - resolve it
        p[k], p[k+1] = p[k+1], p[k]
    return tuple(p)

def simulate_h_flow(N, trials=500, steps=80, start="random", seed=0):
    """Simulate L-flow dynamics with inversion count h(t) as Lyapunov function"""
    print(f"Simulating L-flow for N={N}, {trials} trials, {steps} steps, start='{start}'")
    
    rng = np.random.default_rng(seed)
    H = np.zeros((trials, steps+1), dtype=float)
    
    for t in range(trials):
        if start == "random":
            p = list(range(N))
            rng.shuffle(p)
            p = tuple(p)
        elif start == "worst":
            p = tuple(range(N-1, -1, -1))  # Reverse order - maximum inversions
        else:
            raise ValueError("start must be 'random' or 'worst'")
            
        H[t, 0] = inversion_count(p)
        
        # Simulate L-flow descent
        for s in range(1, steps+1):
            p = local_descent_step(p)
            H[t, s] = inversion_count(p)
            
        if (t + 1) % 100 == 0:
            print(f"  Completed {t+1}/{trials} trials...")
    
    return H

# Validate the bubble-sort lemma
print("L-Flow Time Emergence Analysis")
print("=" * 30)

print("Validating bubble-sort lemma...")
test_perm = (2, 1, 3, 0)  # Example permutation with inversions
h_before = inversion_count(test_perm)

# Apply local descent step
new_perm = local_descent_step(test_perm)
h_after = inversion_count(new_perm)

print(f"Test permutation: {test_perm}")
print(f"Inversions before: {h_before}")
print(f"After local step: {new_perm}")
print(f"Inversions after: {h_after}")
print(f"✓ h decreased by: {h_before - h_after} (should be 0 or 1)")

# Verify maximum inversion counts
max_inversions = {}
for N in [3, 4, 5]:
    worst_perm = tuple(range(N-1, -1, -1))
    max_inv = inversion_count(worst_perm)
    expected_max = N * (N-1) // 2
    max_inversions[N] = max_inv
    print(f"N={N}: max inversions = {max_inv} (expected: {expected_max}) ✓")

print(f"\n✓ Bubble-sort lemma and inversion count validated")
print(f"✓ Maximum inversion counts: {max_inversions}")

## 2. Simulations: monotone descent (N=4 and N=5)
We run many trials from both **random** starts and the **worst** start (reverse order), then plot the mean and interquartile bands to confirm monotone descent of \(h(t)\).

In [ ]:
import matplotlib.pyplot as plt

def plot_h_summary(H, title, outpath):
    """Plot mean and quartile bands for h(t) evolution"""
    t = np.arange(H.shape[1])
    mean_h = H.mean(axis=0)
    q25 = np.quantile(H, 0.25, axis=0)
    q75 = np.quantile(H, 0.75, axis=0)
    
    plt.figure(figsize=(10, 6))
    plt.plot(t, mean_h, 'b-', linewidth=2, label='Mean h(t)')
    plt.fill_between(t, q25, q75, alpha=0.3, color='blue', label='25–75% quartiles')
    
    # Add individual trajectories for first few trials (visual validation)
    for i in range(min(5, H.shape[0])):
        plt.plot(t, H[i], 'k-', alpha=0.2, linewidth=0.5)
    
    plt.xlabel('Time Step')
    plt.ylabel('Inversion Count h(t)')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(outpath, dpi=150, bbox_inches='tight')
    plt.show()
    
    return mean_h, q25, q75

print("\nL-Flow Descent Simulations")
print("-" * 30)

# Comprehensive simulation results
simulation_results = {}

for N in [4, 5]:
    print(f"\nAnalyzing N={N}:")
    
    # Random start simulations
    print("Random starts...")
    H_rand = simulate_h_flow(N, trials=400, steps=80, start='random', seed=123)
    mean_r, q25_r, q75_r = plot_h_summary(H_rand, f'N={N} L-flow: Random Starts', f'./outputs/N{N}_h_flow_random.png')
    
    # Worst case simulations  
    print("Worst case starts...")
    H_worst = simulate_h_flow(N, trials=200, steps=80, start='worst', seed=456)
    mean_w, q25_w, q75_w = plot_h_summary(H_worst, f'N={N} L-flow: Worst Case Starts', f'./outputs/N{N}_h_flow_worst.png')
    
    # Analysis
    initial_random = mean_r[0]
    final_random = mean_r[-1]
    initial_worst = mean_w[0]
    final_worst = mean_w[-1]
    
    random_reduction = (initial_random - final_random) / initial_random
    worst_reduction = (initial_worst - final_worst) / initial_worst
    
    # Monotonicity check
    random_monotonic = np.all(np.diff(mean_r) <= 0.01)  # Allow small numerical fluctuations
    worst_monotonic = np.all(np.diff(mean_w) <= 0.01)
    
    simulation_results[N] = {
        'random_start': {
            'initial_h': float(initial_random),
            'final_h': float(final_random),
            'reduction_fraction': float(random_reduction),
            'monotonic': bool(random_monotonic)
        },
        'worst_start': {
            'initial_h': float(initial_worst),
            'final_h': float(final_worst),
            'reduction_fraction': float(worst_reduction),
            'monotonic': bool(worst_monotonic)
        },
        'max_possible_h': N * (N-1) // 2
    }
    
    print(f"  Random starts: {initial_random:.1f} → {final_random:.1f} ({random_reduction:.1%} reduction)")
    print(f"  Worst starts: {initial_worst:.1f} → {final_worst:.1f} ({worst_reduction:.1%} reduction)")
    print(f"  Monotonicity: Random={random_monotonic}, Worst={worst_monotonic}")

print(f"\n✓ L-flow simulations completed and saved to ./outputs/")

# Summary comparison
print(f"\nL-Flow Performance Summary:")
print("=" * 40)
for N in [4, 5]:
    results = simulation_results[N]
    print(f"N={N} (max h = {results['max_possible_h']}):")
    print(f"  Random: {results['random_start']['reduction_fraction']:.1%} reduction, monotonic: {results['random_start']['monotonic']}")
    print(f"  Worst:  {results['worst_start']['reduction_fraction']:.1%} reduction, monotonic: {results['worst_start']['monotonic']}")

# Time emergence validation
time_emergence_success = all(
    simulation_results[N]['random_start']['monotonic'] and 
    simulation_results[N]['worst_start']['monotonic'] and
    simulation_results[N]['random_start']['reduction_fraction'] > 0.3
    for N in [4, 5]
)

print(f"\n✓ Time emergence validation: {'SUCCESS' if time_emergence_success else 'PARTIAL'}")
if time_emergence_success:
    print("  • h(t) monotonically decreases (Lyapunov property confirmed)")
    print("  • Substantial reduction achieved from both random and worst starts")
    print("  • L-flow provides coherent temporal direction")

## 3. Partial orders: extending the potential to \(\tilde H\)

For a DAG (partial order) \(P\), define \(\mathcal{L}(P)\) = set of **linear extensions** (topological sorts). Fix a reference total order (identity). Define
$$\tilde H(P) = \min_{\sigma\in\mathcal{L}(P)} \mathrm{KT}(\sigma, \mathrm{id}),$$
the minimum Kendall–Tau distance to identity over all linear extensions. \(\tilde H\) equals \(h\) on total orders.

**Proposition (Monotonicity under refinement).** If an update adds a comparability \(u\prec v\) that is consistent (keeps acyclicity), then \(\mathcal{L}(P)\) can only shrink and \(\tilde H\) is **non-increasing**. If all extensions must now resolve an adjacent inversion, \(\tilde H\) strictly decreases.

In [ ]:
import networkx as nx
from itertools import permutations
import json

def kt_distance(order):
    """Kendall–Tau distance to identity for a permutation (tuple)"""
    return inversion_count(order)

def linear_extensions_min_kt(dag_edges, N):
    """Find minimum KT distance among all linear extensions of a DAG"""
    G = nx.DiGraph()
    G.add_nodes_from(range(N))
    G.add_edges_from(dag_edges)
    
    if not nx.is_directed_acyclic_graph(G):
        raise ValueError("Input must be a DAG")
    
    extensions = list(nx.all_topological_sorts(G))
    if not extensions:
        return None
    
    min_kt = min(kt_distance(tuple(topo)) for topo in extensions)
    return min_kt, len(extensions)

print("\nPartial Order Extension Analysis")
print("-" * 35)

# Concrete N=4 example DAG and its refinement
N = 4
print(f"Analyzing DAG refinement for N={N}:")

# Initial partial order: 0<2 and 1<2; nodes 3 is free
P_edges = [(0, 2), (1, 2)]
print(f"Initial DAG edges: {P_edges}")

# Compute Ĥ before refinement
H_before, extensions_before = linear_extensions_min_kt(P_edges, N)
print(f"Linear extensions before: {extensions_before}")
print(f"Ĥ(P) before refinement: {H_before}")

# Add a consistent comparability (2,3) - refines the partial order
P_refined = P_edges + [(2, 3)]
print(f"Refined DAG edges: {P_refined}")

# Compute Ĥ after refinement
H_after, extensions_after = linear_extensions_min_kt(P_refined, N)
print(f"Linear extensions after: {extensions_after}")
print(f"Ĥ(P) after refinement: {H_after}")

# Validate monotonicity property
monotonic = H_after <= H_before
improvement = H_before - H_after

print(f"\nRefinement Analysis:")
print(f"  Ĥ non-increasing: {monotonic} ✓" if monotonic else f"  Ĥ non-increasing: {monotonic} ✗")
print(f"  Improvement: {improvement} inversions")
print(f"  Extension reduction: {extensions_before} → {extensions_after}")

# Additional test cases for robustness
print(f"\nAdditional Refinement Tests:")
test_cases = [
    {
        'name': 'Chain extension',
        'initial': [(0, 1), (1, 2)],
        'refined': [(0, 1), (1, 2), (2, 3)]
    },
    {
        'name': 'Fan convergence', 
        'initial': [(0, 3), (1, 3)],
        'refined': [(0, 3), (1, 3), (0, 1)]
    },
    {
        'name': 'Empty to single edge',
        'initial': [],
        'refined': [(0, 1)]
    }
]

refinement_results = []
for case in test_cases:
    try:
        h_init, ext_init = linear_extensions_min_kt(case['initial'], N)
        h_ref, ext_ref = linear_extensions_min_kt(case['refined'], N)
        
        is_monotonic = h_ref <= h_init
        improvement = h_init - h_ref
        
        result = {
            'name': case['name'],
            'h_before': h_init,
            'h_after': h_ref,
            'monotonic': is_monotonic,
            'improvement': improvement,
            'extensions_before': ext_init,
            'extensions_after': ext_ref
        }
        
        refinement_results.append(result)
        status = "✓" if is_monotonic else "✗"
        print(f"  {case['name']}: Ĥ {h_init} → {h_ref} ({improvement:+d}) {status}")
        
    except Exception as e:
        print(f"  {case['name']}: Error - {e}")

# Overall validation
all_monotonic = all(r['monotonic'] for r in refinement_results)
print(f"\n✓ All refinement tests monotonic: {all_monotonic}")

# Save results
partial_order_results = {
    'main_example': {
        'initial_edges': P_edges,
        'refined_edges': P_refined,
        'h_before': int(H_before),
        'h_after': int(H_after),
        'extensions_before': int(extensions_before),
        'extensions_after': int(extensions_after),
        'monotonic': bool(monotonic),
        'improvement': int(improvement)
    },
    'additional_tests': refinement_results,
    'theoretical_validation': {
        'all_cases_monotonic': bool(all_monotonic),
        'lyapunov_property_confirmed': bool(all_monotonic)
    }
}

with open('./outputs/partial_order_refinement_analysis.json', 'w') as f:
    json.dump(partial_order_results, f, indent=2)

print(f"\n✓ Partial order analysis saved to ./outputs/partial_order_refinement_analysis.json")

# Connection to time emergence
print(f"\nTime Emergence Implications:")
print("=" * 30)
print(f"• Ĥ(P) provides Lyapunov function for partial orders")
print(f"• Refinements (adding comparabilities) are non-increasing")
print(f"• L-flow on partial orders has well-defined temporal direction")
print(f"• Extension validates time emergence beyond total orders")
print(f"• Theoretical foundation: {'SOLID' if all_monotonic else 'NEEDS REVIEW'}")

## 4. Takeaways
- On total orders, **time** corresponds to a monotone descent of **\(h\)** under local L-steps (adjacent swaps that remove inversions).
- On partial orders, the extended potential **\(\tilde H\)** is **non-increasing** under consistent refinements and strictly decreases when an adjacent inversion must be resolved across all linear extensions.
- For **N=5**, these properties continue to hold; the geometry lives in \(V\cong\mathbb{R}^4\) (rank 4), with **time as L-flow** rather than an extra simple root.